In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import xgboost as xgb


In [4]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/unlabeled.csv')

sub_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [5]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [6]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [7]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

In [8]:
#cat_var = cat_var[1:]
#cat_var = cat_var[1:]
cat_var = cat_var + ['weightclass']


In [9]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['integer','numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [10]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [11]:
x_test = test_data.drop('hospital_death',axis=1)

In [12]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [13]:
### impute bmi values based on height and weight
count = 0
bmi_list = []
for i in tqdm(range(x_comb.shape[0])):
    
    if pd.isnull(x_comb["weight"].iloc[i]) or pd.isnull(x_comb["height"].iloc[i]):
        bmi_list.append(x_comb["bmi"].iloc[i])
        continue
    else:
        
        count += 1
        bmi_list.append(x_comb["weight"].iloc[i] / ((x_comb["height"].iloc[i]/100) ** 2))

100%|██████████| 131021/131021 [00:06<00:00, 19186.39it/s]


In [14]:
x_comb['bmi'] = bmi_list

In [15]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [16]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 26/26 [00:03<00:00,  9.22it/s]


In [17]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [18]:
for col in tqdm(new_cont_var):
    x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

100%|██████████| 160/160 [00:00<00:00, 423.32it/s]


In [19]:
count_var = []
for i,col in tqdm(enumerate(new_cont_var + new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

186it [00:14, 13.10it/s]


In [20]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:01<00:00, 68433.03it/s]


In [21]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [22]:
list_illness = ['aids',
                'cirrhosis',
                'diabetes_mellitus',
                'hepatic_failure',
                'immunosuppression',
                'leukemia',
                'lymphoma',
                'solid_tumor_with_metastasis']

In [23]:
inter_cols1 = []

for col in tqdm(list_illness) :
    x_comb['hospital_id'+str(col)] = x_comb['hospital_id'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['gender'+str(col)] = x_comb['gender'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['age'+str(col)] = x_comb['age'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['bmi'+str(col)] = x_comb['bmi'].astype(str) + '_' + x_comb[col].astype(str)
    
    inter_cols1.append('hospital_id'+str(col))
    inter_cols1.append('gender'+str(col))
    inter_cols1.append('age'+str(col))
    inter_cols1.append('bmi'+str(col))

100%|██████████| 8/8 [00:05<00:00,  1.40it/s]


In [24]:
for cols in tqdm(inter_cols1):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 32/32 [00:01<00:00, 21.00it/s]


In [25]:
inter_cols2 = []

for col in tqdm(list_illness) :
    x_comb['age_category'+str(col)] = x_comb['age_category'].astype(str) + '_' + x_comb[col].astype(str)
    inter_cols2.append('age_category'+str(col))

100%|██████████| 8/8 [00:01<00:00,  5.81it/s]


In [26]:
for cols in tqdm(inter_cols2):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 8/8 [00:00<00:00, 32.15it/s]


In [27]:
inter_cols3 = []

for col1 in tqdm(list_illness) :
    for col2 in list_illness:
        if col1 !=col2 :
            x_comb[col1+col2] = x_comb[col1].astype(str) + '_' + x_comb[col2].astype(str)
            inter_cols3.append(col1+col2)

100%|██████████| 8/8 [00:10<00:00,  1.32s/it]


In [28]:
for cols in tqdm(inter_cols3):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 56/56 [00:03<00:00, 30.04it/s]


In [29]:
final_cat_var = new_cat_var + count_var + ['age_category'] + inter_cols1 + inter_cols2 + inter_cols3

##### Target Encoding

In [ ]:
# temp_train = x_train.copy()
# temp_train['hospital_death'] = train_data['hospital_death']

In [ ]:
# cat_count_var = []

# for i,col in tqdm(enumerate(new_cat_var)):
#     mean = temp_train['hospital_death'].mean()
    
#     #Compute the number of values and the mean of each group
#     agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
#     counts = agg['count']
#     means =  agg['mean']
    
#     #Compute the "smoothed" means
#     m=3
#     smooth = (counts * means + m * mean) / (counts + m)
    
#     #Final_val
#     x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
#     cat_count_var.append(str(col)+'count_new')
    

In [ ]:
# # # Remove Features with more than 70 percent missing values
# data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
# data_missing = data_missing.index[data_missing > 0.90]

# x_comb = x_comb.drop(columns = data_missing)

In [ ]:
#cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))

In [ ]:
# for col in tqdm(cat_count_var_new):
#     le = LabelEncoder()
#     x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [ ]:
# aggs = {}

# for col in tqdm(new_cat_var):
#     aggs[col] = ['count', 'nunique']
    

In [ ]:
# for col in tqdm(new_cat_var):
    
#     aggs_temp = aggs.copy()
#     aggs_temp.pop(col)
    
#     agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
#     agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
#     x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
#                     left_on=[col], right_on=[col])
    

In [30]:
x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [31]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [34]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [46]:
xgb_model = xgb.XGBClassifier(
         silent=False,
         n_estimators= 2000,
         max_depth= 30,
         min_child_weight= 2,
         learning_rate=0.01,
         gamma=0.9,                        
         subsample=0.9,
         colsample_bytree=0.8,
         objective= 'binary:logistic',
         nthread= -1,
         scale_pos_weight=1)

In [47]:
xgb.XGBClassifier()


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [48]:
xgb_model.fit(np.array(x1_train), np.array(y1_train))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.9,
              learning_rate=0.01, max_delta_step=0, max_depth=30,
              min_child_weight=2, missing=None, n_estimators=2000, n_jobs=1,
              nthread=-1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.9, verbosity=1)

In [70]:
pred = xgb_model.predict_proba(np.array(test_x))[:,1]

In [71]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = pred

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [72]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/xgboost.csv',index = False)

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 130000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 3000)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

In [ ]:
predictions_10 = predictions / 10.0

In [ ]:
data_sub = x_test[['encounter_id']]

In [ ]:
data_sub['hospital_death'] = predictions_10

In [76]:
a1 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble.csv')
a2 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm.csv')
a3 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm1.csv')
a4 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm2.csv')
a5 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold1.csv')
a6 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold.csv')
a7 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub10_lgb_kfold2.csv')
a8 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/xgboost.csv')
a9 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub11_ensemble_imp1.csv')

In [77]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [78]:
pred_ens = (a1['hospital_death'] + a2['hospital_death'] + a3['hospital_death'] + 
            a4['hospital_death'] + a5['hospital_death'] + a6['hospital_death'] +
            a7['hospital_death'] + a8['hospital_death'] + a9['hospital_death']
           )/9.0
            

In [79]:
sub['hospital_death'] = pred_ens

In [80]:
sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub12_ensemble_xgboost.csv',index = False)

In [ ]:
#train_x['encounter_id'].unique().shape


In [ ]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict_proba(test_x)
preds= pred[:,1]

In [ ]:
sub_data

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = preds

In [ ]:
x_test.head()

In [ ]:
ls

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2.csv',index = False)

In [ ]:
data_sub.shape